In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset,DataLoader
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
from torchvision.models import efficientnet_b3
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import os
import torchvision.utils as vutils
from models import Discriminator, Generator, initialize_weights
from utils import gradient_penalty

In [2]:
date_today = "19 May 23"

In [3]:
path_sport_dataset = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN/Dataset/"
keterangan_data = pd.read_csv(path_sport_dataset+"sports.csv")
sports_chosen = ['arm wrestling', "basketball", "volleyball", "skydiving", "billiards"]
data_chosen = keterangan_data[keterangan_data['labels'].isin(sports_chosen)]

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-4
BATCH_SIZE = 16
IMG_SIZE = 64 #coba 64 aslinya 224
CHANNELS_IMG = 3
Z_DIM = 100
NUM_EPOCHS = 8000
FEATURES_CRITIC = 16
FEATURES_GEN = 16
CRITIC_ITERATIONS = 5
LAMBDA_GP = 10
NUM_CLASSES = len(sports_chosen)
EMBED_SIZE = 5

In [5]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)), #coba ganti  64, harusnya 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def image_to_tensor(file_path,transform):
    img = Image.open(file_path)
    img_tensor = transform(img)
    return img_tensor

data_label = []
images_tensor = []


for file,label in zip(data_chosen['filepaths'], data_chosen['labels']):
    data_label.append(label)
    try:
        images_tensor.append(image_to_tensor(path_sport_dataset+file,transform))
    except:
        pass
stacked_tensor = torch.stack([tensor_img for tensor_img in images_tensor], dim=0)

def label_to_tensor1(labels, data_labels, IMG_SIZE):
    assert type(labels) == list, "Type labels harus list"
    label_channels = len(labels)
    data_labels_tensors = []
    for label in data_labels:
        idx_encode = labels.index(label)
        data_labels_tensors.append(idx_encode)
    return torch.tensor(data_labels_tensors)

labels_tensors = label_to_tensor1(sports_chosen,data_label,stacked_tensor.shape[-1])

In [10]:
len(labels_tensors[labels_tensors==0])

109

In [15]:
JumlahDataPerSport

{'arm wrestling': 109,
 'basketball': 179,
 'volleyball': 152,
 'skydiving': 158,
 'billiards': 155}

In [6]:
# Create a TensorDataset object
dataset = TensorDataset(stacked_tensor, labels_tensors)

# Create a DataLoader object with batch size and shuffle=True
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True)

In [31]:
eksperimen_path = f"C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN/"

In [32]:
# initialize gen and disc, note: discriminator should be called critic,
# according to WGAN paper (since it no longer outputs between [0, 1])
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN, NUM_CLASSES,IMG_SIZE, EMBED_SIZE).to(device)
critic = Discriminator(CHANNELS_IMG, FEATURES_CRITIC,NUM_CLASSES,IMG_SIZE).to(device)
initialize_weights(gen)
initialize_weights(critic)

# initializate optimizer
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))
opt_critic = optim.Adam(critic.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))

In [34]:
pt_disc = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN/Discriminator Model/19 May 23_disc_sports_embed_size_5_epoch_6000.pth"
pt_gen = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN/Generator Model/19 May 23_gen_sports_embed_size_5_epoch_6000.pth"
critic.load_state_dict(torch.load(pt_disc))
gen.load_state_dict(torch.load(pt_gen))
gen.eval()
critic.eval()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(4, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(128, 1, kernel_size=(4, 4), stride=(2, 2))
  )
  (embed): Embedding(5, 4096)

In [35]:
import time
t_start = time.time()
# for tensorboard plotting
fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)

gen.train()
critic.train()

for epoch in range(6000,NUM_EPOCHS):
    for batch_idx, (real, labels) in enumerate(dataloader):
        real = real.to(device)
        cur_batch_size = real.shape[0]
        labels = labels.to(device)
        # Train Critic: max E[critic(real)] - E[critic(fake)]
        # equivalent to minimizing the negative of that
        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn(cur_batch_size, Z_DIM, 1, 1).to(device)
            fake = gen(noise,labels)
            critic_real = critic(real,labels).reshape(-1)
            critic_fake = critic(fake,labels).reshape(-1)
            gp = gradient_penalty(critic,labels, real, fake, device)
            loss_critic = (
                -(torch.mean(critic_real) - torch.mean(critic_fake)) + LAMBDA_GP * gp
            )
            critic.zero_grad()
            loss_critic.backward(retain_graph=True)
            opt_critic.step()

        # Train Generator: max E[critic(gen_fake)] <-> min -E[critic(gen_fake)]
        gen_fake = critic(fake,labels).reshape(-1)
        loss_gen = -torch.mean(gen_fake)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        if batch_idx == 10 and epoch % 50 == 0: 
            with torch.no_grad():
                print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] with  Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}")
                vutils.save_image(real,f"{eksperimen_path}/Sample Image/{date_today} - real samples epoch {epoch} batch {batch_idx}.png", normalize = True)
                fake = gen(noise, labels)
                vutils.save_image(fake, f"{eksperimen_path}/Fake Image/{date_today} - fake samples epoch {epoch} batch {batch_idx}.png",normalize = True)
                
t_end = time.time()
lama = t_end - t_start
print(f"Waktu running {NUM_EPOCHS} menggunakan {device} yaitu {lama} detik")

Epoch [6001/8000] with  Loss D: -23.3875, loss G: 858.4497
Epoch [6051/8000] with  Loss D: -28.4090, loss G: 858.3520
Epoch [6101/8000] with  Loss D: -29.9366, loss G: 870.3267
Epoch [6151/8000] with  Loss D: -23.2066, loss G: 870.5023
Epoch [6201/8000] with  Loss D: -20.7615, loss G: 869.4408
Epoch [6251/8000] with  Loss D: -18.4585, loss G: 865.1625
Epoch [6301/8000] with  Loss D: -27.7621, loss G: 869.7681
Epoch [6351/8000] with  Loss D: -29.6030, loss G: 875.7201
Epoch [6401/8000] with  Loss D: -26.9347, loss G: 876.0898
Epoch [6451/8000] with  Loss D: -23.5447, loss G: 875.1125
Epoch [6501/8000] with  Loss D: -33.2492, loss G: 879.4240
Epoch [6551/8000] with  Loss D: -32.0063, loss G: 878.6902
Epoch [6601/8000] with  Loss D: -30.2753, loss G: 875.5422
Epoch [6651/8000] with  Loss D: -36.8171, loss G: 890.1968
Epoch [6701/8000] with  Loss D: -32.1020, loss G: 890.3405
Epoch [6751/8000] with  Loss D: -32.8735, loss G: 890.0281
Epoch [6801/8000] with  Loss D: -32.6123, loss G: 887.46

In [36]:
#save model
nama_dataset = "sports"
discriminator_path = f"{eksperimen_path}Discriminator Model/{date_today}_disc_{nama_dataset}_embed_size_{EMBED_SIZE}_epoch_{NUM_EPOCHS}.pth"
generator_path = f"{eksperimen_path}Generator Model/{date_today}_gen_{nama_dataset}_embed_size_{EMBED_SIZE}_epoch_{NUM_EPOCHS}.pth"
torch.save(critic.state_dict(), discriminator_path)
torch.save(gen.state_dict(), generator_path)

In [60]:
sports_chosen = ['arm wrestling', "basketball", "volleyball", "skydiving", "billiards"]
def gen_gambar(model,sport_predict,sports_chosen, path, embed_size,NUM_EPOCHS):
    with torch.no_grad():
        jumlah_gambar = 64
        noise = torch.randn(jumlah_gambar, Z_DIM, 1, 1).to(device)
        idx_encode = sports_chosen.index(sport_predict)
        encode = [idx_encode for _ in range(jumlah_gambar)]
        labels = torch.tensor(encode).to(device)
        pic_predict = model(noise,labels)
        vutils.save_image(pic_predict, f"{path} {date_today} - Hasil predict 16 gambar {sport_predict} embed size {embed_size} epoch {NUM_EPOCHS}.png", normalize = True)

In [61]:
for sport in sports_chosen:
    path_test = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN/Test Image/"
    gen_gambar(gen,sport, sports_chosen, path_test, EMBED_SIZE, NUM_EPOCHS)
    

torch.Size([64])
torch.Size([64, 100, 1, 1])
torch.Size([64])
torch.Size([64, 100, 1, 1])
torch.Size([64])
torch.Size([64, 100, 1, 1])
torch.Size([64])
torch.Size([64, 100, 1, 1])
torch.Size([64])
torch.Size([64, 100, 1, 1])


Waktu running 300 epoch pertama : 1199.4615950584412 
waktu running 300 epoch kedua : 1112.2095129489899 
kedua running 4000 epoch ketiga : 13179.149538993835

In [12]:
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

3.660833333333333